# Calories RAG - Open Source Model Version

In [1]:
import os
import glob
from dotenv import load_dotenv
from huggingface_hub import login

## Load HuggingFace and login

In [2]:
load_dotenv(override=True)
hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Setup Mongo connection

In [3]:
import json
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer

In [4]:
MONGO_DB_USER = os.getenv('MONGO_DB_USER')
MONGO_DB_PASSWORD = os.getenv('MONGO_DB_PASSWORD')
MONGO_DB_CLUSTER_NAME = os.getenv('MONGO_DB_CLUSTER_NAME')

DB_NAME = 'nutritional_rag'
COLLECTION_NAME = 'food'

uri = f"mongodb+srv://{MONGO_DB_USER}:{MONGO_DB_PASSWORD}@{MONGO_DB_CLUSTER_NAME}.i1ndjzi.mongodb.net/?retryWrites=true&w=majority&appName={MONGO_DB_CLUSTER_NAME}"

client = MongoClient(uri)
collection = client[DB_NAME][COLLECTION_NAME]

## Setup search index function

In [5]:
transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
# Define a function to run vector search queries
def get_query_results(query):
  """Gets results from a vector search query."""

  query_embedding = transformer.encode(query).tolist()
  pipeline = [
      {
            "$vectorSearch": {
              "index": "food_vector_index",
              "queryVector": query_embedding,
              "path": "embedding",
              "exact": True,
              "limit": 3
            }
      }, {
            "$project": {
              "_id": 0,
              "text": 1
         }
      }
  ]

  results = collection.aggregate(pipeline)

  array_of_results = []
  for doc in results:
      array_of_results.append(doc)
  return array_of_results

## Create query function to Open source model

In [7]:
import json
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline

In [8]:
class AnswerFormat(BaseModel):
    protein: float
    carbohydrates: float
    fats: float
    calories: float
    sugar: float
    fiber: float

In [9]:
def get_nutritional_data(food):
  context = get_query_results(food)
  context_string = " - ".join([doc["text"] for doc in context])

  prompt = f"""
    Get the nutritional data of the following food ingredient: {food}. 
    Answer the question based only on the following context: {context_string}. 
    Reply only with a JSON in the following json schema: {AnswerFormat.model_json_schema()}.
  """

  hf_pipeline = pipeline('text-generation', model='Qwen/Qwen2.5-3B-Instruct')

  # Create a character level parser and build a transformers prefix function from it
  parser = JsonSchemaParser(AnswerFormat.model_json_schema())
  prefix_function = build_transformers_prefix_allowed_tokens_fn(hf_pipeline.tokenizer, parser)

  # Call the pipeline with the prefix function
  output_dict = hf_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function, max_new_tokens=100)

  # Extract the results
  result = output_dict[0]['generated_text'][len(prompt):].replace('\n', '')

  return json.loads(result)

In [10]:
response = get_nutritional_data("milk")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [11]:
response

{'protein': 3.28,
 'carbohydrates': 4.65,
 'fats': 3.66,
 'calories': 64,
 'sugar': 0.0,
 'fiber': 0.0}